# Implementation of Simple NN based Ranking Model using Model Similarity feature of Word Embeddings.

```/* @Author: Jagan Kaartik */```

In [ ]:
import os
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from collections import defaultdict

In [ ]:
os.getcwd()

In [ ]:
path = "./Dataset"

In [ ]:
filelist = os.listdir(path)
filelist

In [ ]:
filelist.pop(0)

In [ ]:
corpus = []
for i in filelist:
     with open("{}".format(i)) as f_input:
            corpus.append(f_input.read())

In [ ]:
len(corpus)

In [ ]:
# Check
corpus[0]

## Pre-Processing Corpus Data 

In [ ]:
newpath = "./Dataset"
os.chdir(newpath)

In [ ]:
# Remove \n in Corpus Txt
for c in range(0,len(corpus)):
    corpus[c] = re.sub('\n','',corpus[c])

## Lemmatization and Removal of Stop Words from Corpus

In [ ]:
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer() 

for c in range(0,len(corpus)):
    word_list = nltk.word_tokenize(corpus[c])
    filtered_sentence = [w for w in word_list if not w in stop_words]
    corpus[c] = ' '.join([lemmatizer.lemmatize(w) for w in filtered_sentence])

In [ ]:
corpus[0]

### Writing Lemmatized Corpus to File for Efficient Retreival 

In [ ]:
path = "./Dataset/Lemmatized Corpus/"
os.chdir(path)

In [ ]:
for c in range(0,len(corpus)):
    with open('doc{}.txt'.format(c+1), 'w') as f:
        f.write("%s\n" % corpus[c])

### Inverted - Index

In [ ]:
class InvertedIndex: 
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.index = defaultdict(list)
        self.documents = {}
        self.unique_id = 0
 
    def termLookup(self, word):
        word = word.lower()
        return [self.documents.get(id, None) for id in self.index.get(word)]
    
    def retIndex(self):           
        return self.index.items()
 
    def addTerm(self, document):
        for token in [t.lower() for t in nltk.word_tokenize(document)]:
            if self.unique_id not in self.index[token]:
                self.index[token].append(self.unique_id)
 
        self.documents[self.unique_id] = document
        self.unique_id += 1           

invObj = InvertedIndex(nltk.word_tokenize)

### Adding Corpus Data to Inverted Index

In [ ]:
for c in range(0,100):
    invObj.addTerm(corpus[c])

In [ ]:
res = invObj.termLookup("cancer")
print(len(res))

### View Inverted Index

In [ ]:
invIndex = invObj.retIndex()

for k,v in invIndex:
    print("{} ---> {}".format(k,v)) 

In [ ]:
len(invIndex)

### Lookup for Terms in Inverted Index

In [ ]:
term = "cancer"
lookupResult = invObj.termLookup(term)

In [ ]:
print("No of Documents with Term {} is {} ".format(term,len(lookupResult)))

### Saving Invertex Index to CSV

In [ ]:
import csv
w = csv.writer(open("InvertedIndex.csv", "w"))
for key, val in invIndex:
    w.writerow([key, val])

### Size of Invertex Index with 100 corpus documents.

In [ ]:
print(len(invIndex))

### Load Lemmatized Corpus

In [ ]:
path = "./Dataset/Lemmatized Corpus/"
os.chdir(path)

In [ ]:
filelist = os.listdir(path)
filelist

In [ ]:
corpus = []
for i in filelist:
     with open("{}".format(i)) as f_input:
            corpus.append(f_input.read())

In [ ]:
#Lemmatized Corpus
corpus[0]

### Convert Corpus to Vectors

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize 

In [ ]:
def processCorpus(train):
    traincorp = []
    for i in train:
        traincorp.append(word_tokenize(i))
    return traincorp

In [ ]:
train = corpus

In [ ]:
traincorp = processCorpus(train)

In [ ]:
modelW2V = Word2Vec(traincorp, min_count=1)

In [ ]:
modelW2V.save('modelfull.bin')

* Refer : https://machinelearningmastery.com/develop-word-embeddings-python-gensim/ 
* Refer : https://www.geeksforgeeks.org/tokenize-text-using-nltk-python/
* Refer : https://stackoverflow.com/questions/55713132/how-to-tokenize-a-list-of-lists-in-python

### Model Based Similarity

##### Score of using our Model to compare Cancer with similar words such as Ovarian, Tumor

In [ ]:
modelW2V.wv.similarity('cancer', 'tumor') 

In [ ]:
modelW2V.wv.similarity('cancer','ovarian')

##### Score of using our Model to compare Cancer with dissimilar words such as Cloud

In [ ]:
modelW2V.wv.similarity('cancer','cloud')

##### Finding top 10 most similar words to word Cancer in our dataset

In [ ]:
v1 = "cancer"
modelW2V.wv.most_similar(positive=v1)

## Doc2Vec Model for Finding out document similarity

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(corpus)]

### Traning the Model

In [ ]:
max_epochs = 10
vec_size = 20 
alpha = 0.025

In [ ]:
model = Doc2Vec(size=vec_size,alpha=alpha, min_alpha=0.00025,min_count=50,dm =1)

In [ ]:
model.build_vocab(tagged_data)

In [ ]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

In [ ]:
model.save("min50.model")

### Cosinie Similarity between Documents

In [ ]:
from scipy import spatial

In [ ]:
vec1 = model.infer_vector(corpus[1].split())
vec2 = model.infer_vector(corpus[3].split())

In [ ]:
similairty = spatial.distance.cosine(vec1, vec2)
similairty

In [ ]:
new_sentence = "i love dogs".split(" ") 

In [ ]:
model.docvecs.most_similar(positive=[model.infer_vector(new_sentence)],topn=5)

In [ ]:
corpus[5868]

In [ ]:
model = Doc2Vec.load("d2v.model")

In [ ]:
query = "i love dogs".split(" ") 

In [ ]:
model.docvecs.most_similar(positive=[model.infer_vector(query)],topn=5)

In [ ]:
# Corpus Regarding Euthanesia, Animal Health, Veterinarian etc. 
# These WORDS are Similar in semantic similarity with Dogs!
corpus[638]

#### Benefits of Semantic Similarity based Ranking is, documents related in semantic similarity with the query is returned. Using this to initialty filter results (documents) would be ideal.

In [ ]:
query = "Healthcare Research".split(" ") 

In [ ]:
model.docvecs.most_similar(positive=[model.infer_vector(query)],topn=5)

In [ ]:
# Doc with highest similarity
corpus[5508]

In [ ]:
query = "safety".split(" ") 
model.docvecs.most_similar(positive=[model.infer_vector(query)],topn=5)

In [ ]:
# Doc with highest similarity
corpus[107]

In [ ]:
newmodel = Doc2Vec.load("d2vmin50.model")

In [ ]:
query = "dog".split(" ") 
newmodel.docvecs.most_similar(positive=[model.infer_vector(query)],topn=5)

In [ ]:
# Doc with highest similarity
corpus[748]